This notebook outlines the following steps for the `tgviz` app.  

1. Read 55 AISNPs from Kidd manuscript and save to file. 
2. Download the 1000 Genomes Project `.bcf` files.
3. Use `cyvcf2` to write one `.vcf` file with all 1000 Genomes Project samples' genotypes for each of the 55 AISNPs.
4. Read 128 AISNPs from the Seldin Lab manuscript and save to file.
5. Use `cyvcf2` to write one `.vcf` file with all 1000 Genomes Project samples' genotypes for each of the 128 AISNPs.

Run this notebook from the `notebooks/` directory, or you will need to change the filepaths.

In [1]:
import os
import pandas as pd
from cyvcf2 import VCF, Writer

import warnings
warnings.filterwarnings('ignore')

# Read 55 AISNPs from [Kidd et al.](https://www.sciencedirect.com/science/article/pii/S1872497314000039) Table 1

I used `pd.read_clipboard()` and copy pasted from the link to create `data/kidd_55_AISNPs.txt`,  but I commited the file to the GitHub repository.

In [2]:
df55 = pd.read_csv('../data/Kidd_55_AISNPs.txt', sep='\t')

In [3]:
df55.head()

,dbSNP rs#,Chr,Build 37 nt position,73-population Fst
0,rs3737576,1,"101,709,563",0.44
1,rs7554936,1,"151,122,489",0.39
2,rs2814778,1,"159,174,683",0.82
3,rs798443,2,"7,968,275",0.34
4,rs1876482,2,"17,362,568",0.75


The following changes were made to Table 1 from the manuscript.


| dbSNP rs# | chr, Kidd et al. | chr, new |  
|---|---|---|
| rs870347 | 6 | 5 |  
| rs192655 | 7 | 6 |  
| rs917115 | 8 | 7 |

# Download the 1000 genomes .bcfs  

At the command line, type the following, **these took over an hour to download for me.**

```shell
mkdir 1kg_bcfs
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/supporting/bcf_files/* -P 1kg_bcfs
```

# Write a new 1kG .vcf file with the 55 Kidd AISNPs.

In [4]:
# replace /Users/kevin/projects/tgviz with /path/to/1kg_bcfs
bcf_path = '/Users/kevin/projects/tgviz/1kg_bcfs/'

bcf_fname = 'ALL.chr{}.phase3_shapeit2_mvncall_integrated_v5.20130502.genotypes.bcf'.format(1)
bcf = VCF(os.path.join(bcf_path, bcf_fname))
w = Writer('../data/Kidd.55AISNP.1kG.vcf', bcf)

for i, aim in df55.iterrows():
    chrom = str(aim['Chr'])
    pos = str(aim['Build 37 nt position'])
    pos = pos.replace(',', '')
    rsid = str(aim['dbSNP rs#'])
    bcf_fname = 'ALL.chr{}.phase3_shapeit2_mvncall_integrated_v5.20130502.genotypes.bcf'.format(chrom)
    bcf = VCF(os.path.join(bcf_path, bcf_fname))
    for variant in bcf("{}:{}-{}".format(chrom, pos, pos)):
        if variant.POS == int(pos):
            w.write_record(variant)
w.close(); bcf.close()   

# Now, read 128 AISNPs from the Seldin Lab manuscript

In [5]:
df128 = pd.read_csv('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3073397/bin/NIHMS65513-supplement-S2.txt', 
                    sep='\t', skiprows=1)

In [6]:
df128.to_csv('../data/Seldin_128_AISNPs.txt', index=False, sep='\t')

In [7]:
df128.head()

,NCBI SNP Reference,Assay ID,Strand,VIC,FAM,Context Sequence,Chr,Celera ID,NCBI Assembly Build Number,Location on NCBI Assembly,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,1
0,rs731257,C_____14517_10,Forward,A,G,AAGTTGCAATATGGCAAAACCTGTA[A/G]GAGATACAATTTGTGA...,7,hCV14517,36,12635776,NaN,NaN,NaN,NaN,143
1,rs2946788,C____302128_10,Reverse,G,T,TGAAAAGCTTTAGAAGAAAAAAGCT[G/T]TGTGGCTATTGAGTTT...,11,hCV302128,36,23967106,NaN,NaN,NaN,NaN,167
2,rs3793451,C____320568_10,Reverse,C,T,GGTTATCATGGCTGCCCTCTCACTT[C/T]TTCAGAGACATGTGTT...,9,hCV320568,36,70849100,NaN,NaN,NaN,NaN,38
3,rs10236187,C____328256_10,Forward,A,C,GAACGGCAGACAAAGCCTCACATTA[A/C]GCATCTCTTTAGTAAA...,7,hCV328256,36,139093846,NaN,NaN,NaN,NaN,91
4,rs1569175,C____441412_10,Reverse,C,T,TTCTTCCTCATCATCATCGAAGTTA[C/T]TTATTGATACCTCTTC...,2,hCV441412,36,200730199,NaN,NaN,NaN,NaN,50


In [9]:
df128[['Chr', 'Location on NCBI Assembly']].to_csv('../data/Seldin_128_AISNPs.grch36.txt', index=False, sep='\t')

# Remap from GRCh36 -> GRCh37  
You can use [this tool](https://www.ncbi.nlm.nih.gov/genome/tools/remap) and `Seldin_128_AISNPs.grch36.txt` (exported in previous cell) to remap from GRChr36 to GRChr37. 

...converting to GRCh37...

# Read the converted Seldin Lab 128 AISNPs

In [10]:
df128_hg19 = pd.read_csv('../data/report_Seldin_128_AISNPs.grch36.txt.xls', sep='\t')

In [11]:
df128_hg19.head()

,#feat_name,source_int,mapped_int,source_id,mapped_id,source_length,mapped_length,source_start,source_stop,source_strand,source_sub_start,source_sub_stop,mapped_start,mapped_stop,mapped_strand,coverage,recip,asm_unit,Unnamed: 18
0,Line:2,1,1,7,7,1,1,12635776,12635776,+,12635776,12635776,12669251,12669251,+,1.0,First Pass,Primary Assembly,NaN
1,Line:3,1,1,11,11,1,1,23967106,23967106,+,23967106,23967106,24010530,24010530,+,1.0,First Pass,Primary Assembly,NaN
2,Line:4,1,1,9,9,1,1,70849100,70849100,+,70849100,70849100,71659280,71659280,+,1.0,First Pass,Primary Assembly,NaN
3,Line:5,1,1,7,7,1,1,139093846,139093846,+,139093846,139093846,139447377,139447377,+,1.0,First Pass,Primary Assembly,NaN
4,Line:6,1,1,2,2,1,1,200730199,200730199,+,200730199,200730199,201021954,201021954,+,1.0,First Pass,Primary Assembly,NaN


# Write another new 1kG .vcf file, this time with the 128 Seldin AISNPs.

In [12]:
# replace /Users/kevin/projects/tgviz with /path/to/1kg_bcfs
bcf_path = '/Users/kevin/projects/tgviz/1kg_bcfs/'
bcf_fname = 'ALL.chr{}.phase3_shapeit2_mvncall_integrated_v5.20130502.genotypes.bcf'.format(1)
bcf = VCF(os.path.join(bcf_path, bcf_fname))

w = Writer('../data/Seldin.128AISNP.1kG.vcf', bcf)
for i, aim in df128_hg19.iterrows():
    chrom = aim['source_id']
    pos = aim['mapped_start']
    bcf_fname = 'ALL.chr{}.phase3_shapeit2_mvncall_integrated_v5.20130502.genotypes.bcf'.format(chrom)

    bcf = VCF(os.path.join(bcf_path, bcf_fname))
    for variant in bcf("{}:{}-{}".format(chrom, pos, pos)):
        if variant.POS == int(pos):
            w.write_record(variant)
w.close(); bcf.close()

# There should be two `.vcf` files in the `data/` directory.

* `data/Kidd.55AISNP.1kG.vcf` 
* `data/Seldin.128AISNP.1kG.vcf`